In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import warnings
from ipywidgets import HTML, Output, Layout, Button, VBox

from packages.parquet_cached_df import ParquetCachedDF
from packages.widgets_recipe import items_list
from packages.widgets import DataEntryWidget, DateInput

DEBUG = False
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df_path = "./weight_loss_dfs/jordan_df.pqt"
#df_path_tmp = "./weight_loss_dfs/jordan_df_copy.pqt"

In [ ]:
style = """
    <style>
       .jupyter-widgets-output-area .output_scroll {
            height: unset !important;
            border-radius: unset !important;
            -webkit-box-shadow: unset !important;
            box-shadow: unset !important;
        }
        .jupyter-widgets-output-area  {
            height: auto !important;
        }
    </style>
    """
display(HTML(style))

In [ ]:
class DataEntry:
    def __init__(
        self,
        df_manager,
        date_entry_widget,
        data_entry_recipe,
        output_obj,
        debug_output,
        form_layout=None
    ):
        
        self.df_manager = df_manager
        self.data_entry_recipe = data_entry_recipe
        for item in self.data_entry_recipe:
            item["widget"] = item["widget_class"](
                label=item["col_title"],
                units=item["units"],
                **item["widget_class_args"],
            )
        self.date_entry_widget = date_entry_widget
        self.output_obj = output_obj
        self.debug_output = debug_output
        
        if form_layout is None:
            self.form_layout = Layout()
        else:
            self.form_layout = form_layout
            
        self.submit_button = Button(
            description="Submit",
            disabled=False,
            button_style="success",
            tooltip="submit form",
            icon="check",
            layout=Layout(
                width="max-content"
            )
        )
        
        # initialize data entry widgets
        def handle_submit(e):
            self.add_row(e)
        self.submit_button.on_click(handle_submit)

        self.delete_button = Button(
            description=f"Delete Row {self.date_entry_widget.value}",
            disabled=True,
            button_style="danger",
            tooltip="delete row for date",
            icon="trash-alt",
            layout=Layout(
                width="max-content"
            )
            
        )
        def handle_delete(e):
            self.delete_row(e)
        self.delete_button.on_click(handle_delete)
            
        def handle_date_entry(e):
            self.set_defaults(date=e["new"])
        self.date_entry_widget.observe(handle_date_entry, names="value")
        
        form_children = [self.date_entry_widget.to_box()]
        form_children += [item['widget'].to_box() for item in self.data_entry_recipe]
        form_children += [self.submit_button, self.delete_button]
        
        self.form = VBox(children=form_children, layout=self.form_layout)
        self.set_defaults()
        self.update_display(msg="Select a date and enter your values...") 
        
    def add_row(self, e):
        msg = ""
        df = None
        idx = str(self.date_entry_widget.value)
        extras=[]
        
        if idx == "None":
            msg = "Must enter a date!"
            
        else:
            row = {item["widget"].get_col_title(): item["widget"].value for item in self.data_entry_recipe}
            
            if DEBUG:
                with self.debug_output:
                    display(row)

            for key in row.keys():
                if not isinstance(row[key], bool):
                    if row[key] == 0:
                        row[key] = np.nan

            df = self.df_manager.get_df()
            if df is None:
                df = pd.DataFrame(
                    data=row,
                    index=[idx]
                )
                msg = "Success! New dataset created!"
                
            elif idx in df.index:

                def overwrite(e):
                    for k, v in row.items():
                        df.loc[idx, k] = v
                    self.cache_df(df)
                    self.update_display(msg="Row updated!")

                overwrite_btn = Button(
                    description=f"Overwrite data for {str(idx)}",
                    disabled=False,
                    button_style="warning",
                    tooltip="overwrite row",
                    icon="check",
                    layout=Layout(
                        width="max-content"
                    )
                )
                overwrite_btn.on_click(overwrite)
                msg = "Data already exists for this date. Overwrite data?"
                extras.append(overwrite_btn)

            else:
                df = df.append(pd.Series(row, name=idx))
                msg = "Successfully added row!"
                
            self.cache_df(df)

        self.update_display(msg=msg, extras=extras)
        
    def delete_row(self, e):
        self.delete_button.disabled = True
        idx = str(self.date_entry_widget.value)
        extras=[]
        
        def delete(e):
            msg = ""
            idx = str(self.date_entry_widget.value)
            extras = []

            df = self.df_manager.get_df()
            if df is None:
                msg = "No data exists yet!"
            elif idx not in df.index:
                msg = f"Could not locate data for {idx}"
            else:
                df = df.drop(labels=idx, axis='index')
                msg = "Row was succesfully deleted"
                self.cache_df(df)
                self.update_display(msg=msg)
            
        delete_btn = Button(
            description=f"Delete data for {str(idx)}",
            disabled=False,
            button_style="warning",
            tooltip="delete row",
            icon="exclamation-triangle",
            layout=Layout(
                width="max-content"
            )
        )
        delete_btn.on_click(delete)
        msg = "Confirm deletion"
        extras.append(delete_btn)
        
        self.update_display(msg=msg, extras=extras)
    

    def update_display(self, msg, extras=[]):
        with self.output_obj:
            if not DEBUG:
                self.output_obj.clear_output()
            display(HTML(msg))
            for extra in extras:
                display(extra)
            self.set_defaults(date=self.date_entry_widget.value)
            display(self.form)
            display(self.df_manager.get_df())
            
    def cache_df(self, df):
        self.df_manager.set_df(df.sort_index(ascending=False))
        self.df_manager.save_df()
        
    def set_widget_with_most_recent_valid_value(self, df, col_title, before_date):
        
        if DEBUG:
            with debug_output:
                display("set_widget_with_most_recent_valid_value")
                display("col_title:", col_title)
                display("before_date:", before_date)
        
        # find the applicable recipe entry
        recipe_entry = None
        for item in self.data_entry_recipe:
            
            if DEBUG:
                with debug_output:
                    display(item["widget"].get_col_title())
                    
            if item["widget"].get_col_title() == col_title:
                recipe_entry = item
                break
                
        if recipe_entry is None:
            raise ValueError(f"{col_title} is an invalid column name")
        else:
            df_t = df[df.index < str(before_date)]
            if col_title not in df_t.columns or len(df_t[col_title].dropna()) == 0:
                val = recipe_entry["default_val"]
            else:
                val = df_t[col_title].dropna()[0]
            recipe_entry["widget"].value = recipe_entry["type"](val)
        
                

    def set_defaults(self, date=None):
        df = self.df_manager.get_df()
        
        if date is None:
            date = datetime.date.today()
        if df is not None and str(date) in df.index:
            
            self.delete_button.disabled = False
            self.delete_button.description = f"Delete data for {self.date_entry_widget.value}"
            
            df_row = df.loc[str(date)]

            for item in self.data_entry_recipe:
                try:
                    item["widget"].value = item["type"](df_row[item["widget"].get_col_title()])
                except:
                    warnings.warn(f"Could not get default value for {item['col_title']}")
                    item["widget"].value = item["type"](item["default_val"])
        else:
            self.delete_button.disabled = True
            self.delete_button.description = f"Delete data for {self.date_entry_widget.value}"
            for item in self.data_entry_recipe:
                item["widget"].value = item["type"](item["default_val"])
            self.set_widget_with_most_recent_valid_value(
                df=df,
                col_title="Target Calories (kcal)",
                before_date=date
            )
            #datetime.datetime.strptime(str(date), "%Y-%m-%d"),

In [ ]:
# initial setup
df_manager = ParquetCachedDF(file_path=df_path)
out = Output()
debug_out = Output()

display(out)
display(debug_out)

date = DateInput(label="Date", units=None)
engine = DataEntry(
    df_manager=df_manager,
    date_entry_widget=date,
    data_entry_recipe=items_list,
    output_obj=out,
    debug_output=debug_out,
)

In [ ]:
raise ValueError("stop here!")

In [ ]:
# fix df
#df = df_manager.get_df()

In [ ]:
#list(df.columns)

In [ ]:
#df = df.rename(columns={"Yoga": "Stretch"})

In [ ]:
#df.columns

In [ ]:
#df.head()

In [ ]:
#df = df[
#    [
#        'Weight (lb)',
#        'Body Fat (%)',
#        'BMI',
#        'Skeletal Muscle Mass (lb)',
#        'Bone Mass (lb)',
#        'Body Water (%)',
#        'Waist (in)',
#        'Belly (in)',
#        'Hips (in)',
#        'Chest (in)',
#        'Bicep (in)',
#        'Thigh (in)',
#        'Calf (in)',
#        'Target Calories (kcal)',
#        'Consumed Calories (kcal)',
#        'Active Calories (kcal)',
#        'Resting Heart Rate (bpm)',
#        'Workout',
#        'Cardio',
#        'Yoga',
#        'Meditate',
#        'Mile Time (min)',
#        'Mode',
#        #'Weight',
#        #'Body Fat',
#        #'Skeletal Muscle Mass',
#        #'Bone Mass',
#        #'Body Water',
#        #'Waist',
#        #'Belly',
#        #'Hips',
#        #'Bicep',
#        #'Chest',
#        #'Thigh',
#        #'Calf',
#        #'Target Calories',
#        #'Consumed Calories',
#        #'Active Calories',
#        #'Resting Heart Rate',
#        #'Mile Time'
#    ] 
#]

In [ ]:
#df_manager.set_df(df.sort_index(ascending=False))
#df_manager.save_df()

In [ ]:
#df = df_manager.get_df()

In [ ]:
#df.columns